# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [28]:
###MAKE SURE NO OUTSIDE VARIABLES BEING USED

def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    demand=nx.get_node_attributes(graph,'demand')
    ##create a copy of a graph
    G_temp = nx.DiGraph(graph)
    G_temp.add_node('s')
    G_temp.add_node('t')
    ##connect sources to big source and sinks to big sink
    for nodes in graph.nodes():
        node_demand = demand[nodes]
        if node_demand < 0 : 
            G_temp.add_edge('s',nodes,capacity=abs(node_demand))
        if node_demand > 0 : 
            G_temp.add_edge(nodes,'t',capacity=abs(node_demand))
            
    ##initialize to zero, all the flows
    flow = dict()
    for nodes in G_temp.nodes():
        temp = dict()
        for key,value in G_temp[nodes].iteritems():
            temp[key] = 0 
        flow[nodes] = temp
    
    ##returns a residual graph of a graph
    def residual(graph, flow, source):
        G = nx.DiGraph()
        ##adding nodes
        for nodes in graph.nodes():
            G.add_node(nodes)
        visited = dict()
        ##key exists if visited, value is origin
        ##using BFS-like method to populate the edges starting from the source to avoid backwards edges
        visited[source] = source
        queue = []
        queue.append(source)
        ##adding edges, not including backwards edges
        while ((len(queue)>0)):
            visit = queue.pop(0)
            for neighbor, capacity in graph[visit].iteritems():
                ##not visited and has capacity across the path
                if (neighbor not in visited):
                    queue.append(neighbor)
                    visited[neighbor] = visit     
                ###add forward edges, capacity - flow
                if (not G.has_edge(neighbor,visit)):
                    G.add_edge(visit,neighbor,
                               capacity = graph[visit][neighbor]['capacity'] - \
                                   flow[visit][neighbor],direction='forward')
                    ##only if it has flow already, add backwards edges, -flow
                    if flow[visit][neighbor] > 0: 
                        G.add_edge(neighbor,visit,
                                   capacity = flow[visit][neighbor],direction='backward')
        return G
    
    ###BFS search, terminates once sink is found 
    def BFS(graph, source, sink):
        visited = dict()
        ##key exists if visited, value is origin
        visited[source] = source
        #### Using list as a queue, NOTE O(N) access unlike actual queue but we can't import pacakages.
        queue = []
        queue.append(source)

        ###while we have stuff in our queue
        just_stop = False
        while ((len(queue)>0) & (just_stop is False)):
            visit = queue.pop(0)
            for neighbor, capacity in graph[visit].iteritems():
                ##found s-t path and is reachable i.e. has positive capacity 
                if (neighbor == sink) & (capacity['capacity']>0):
                    visited[neighbor] = visit
                    just_stop = True
                    break
                ##not visited and has capacity across the path
                if (neighbor not in visited) &(capacity['capacity']>0):
                    queue.append(neighbor)
                    visited[neighbor] = visit     
        return visited
    
    ##given BFS output, resolve path and minimum capacity
    def resolve_path(graph,min_capacity, path, path_out,origin):
        path_out.append(origin)
        if path[origin]==origin:
            return path_out, min_capacity 
        min_capacity = min(graph[path[origin]][origin]['capacity'],min_capacity)
        return resolve_path(graph,min_capacity,path,path_out,path[origin])
        
    while True:
        residual_graph = residual(G_temp,flow,'s')
        s_t_path = BFS(residual_graph,'s','t')
        if('t' in s_t_path):
            ##resolve path 
            path,capacity= resolve_path(residual_graph,float('inf'),s_t_path,[],'t')
            for idx in range(len(path)-1):
                dest=path[idx]
                org=path[idx+1]
                if(residual_graph[org][dest]['direction']=='forward'):
                    flow[org][dest] = flow[org][dest] + capacity
                else:
                    flow[org][dest] = flow[org][dest] - capacity
        else: break 
    
    ##delete big sink and big source
    del flow['s']
    for field in flow.keys():
        if ('t' in flow[field]): del flow[field]['t']
    
    return flow

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [29]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    d = dict()
    ##init
     
    for state,neighbors in flow.iteritems():
        for neighbor, flow_to in neighbors.iteritems():
            if state not in d:
                d[state] =0
            if neighbor not in d:
                d[neighbor] =0
            ### add flows that come in and subtract flows that go out
            d[neighbor]  = d[neighbor] +flow_to
            d[state] = d[state] - flow_to
    return d

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [30]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
